https://github.com/PacktPublishing/Python-Social-Media-Analytics/blob/5886b29587054526920e40b54ea630de5ed9d365/Chapter03/Chapter3.ipynb


https://developers.facebook.com/docs/graph-api/explorer/

https://developers.facebook.com/docs/places/search

https://developers.facebook.com/docs/graph-api/reference/page/ratings/

https://developers.facebook.com/tools/explorer/

In [1]:
#!pip install facepy
#!pip install pony

In [2]:
import requests
import json
import time
import db_reacao
from datetime import datetime
from facepy import GraphAPI
from pony import orm

In [73]:
def insert_facebook(cliente, page_id, query_time_post, query_time_comm, get_feed):
    global erros_insert_comment
    global insert_comment
    
    post={}
    comm={}
    reac={}
    reac_sentimento = {"SAD":-1,"ANGRY":-1,"HAHA":1,"LOVE":1,"WOW":1,"LIKE":1}
    
    if get_feed==True:
        data_post = graph.get('v2.12/' + page_id + '/feed?')
        for p in range(0,len(data_post["data"])):
            post = data_post["data"][p]
            created_time_post = datetime.strptime(post.get("created_time").split("T")[0], '%Y-%m-%d')
            if created_time_post>=query_time_post:
                data_comm = graph.get('v2.12/'+ post.get("id") +'?fields=comments,reactions')
                if data_comm.get("comments")!=None:                
                    for c in range(0,len(data_comm["comments"]["data"])):
                        comm = data_comm["comments"]["data"][c]
                        created_time_comm = datetime.strptime(comm.get("created_time").split("T")[0], '%Y-%m-%d')
                        if created_time_comm>=query_time_comm:
                            try:
                                insert_facebook(cliente, comm.get("id"), query_time_post, query_time_comm,False)
                                db_reacao.add_reacao_face(cliente,2,comm["message"], 0,int(time.mktime(created_time_comm.timetuple())),str(comm["id"]))
                                print("Registro inserido".center(110,"*"))
                                print("Comentário: " + comm["message"])
                                insert_comment += 1
                            except Exception as e:
                                erros_insert_comment += 1
                                print("Registro com erro".center(110,"*"))
                                print("Comentário: " + comm["message"])
                                print("Erro: " + str(e))  
                if data_comm.get("reactions")!=None:
                    for c in range(0,len(data_comm["reactions"]["data"])):
                        reac = data_comm["reactions"]["data"][c]
                        try:            
                            db_reacao.add_reacao_face(cliente,3,reac["type"], reac_sentimento.get(reac["type"],0),int(time.mktime(datetime.now().timetuple())),str(post["id"])+"_"+str(reac["id"]))
                            print("Registro inserido".center(110,"*"))
                            print("Reação: " + reac["type"])
                            insert_comment += 1
                        except Exception as e:
                            erros_insert_comment += 1
                            print("Registro com erro".center(110,"*"))
                            print("Reação: " + reac["type"])
                            print("Erro: " + str(e))    
    else:
        data_comm = graph.get('v2.12/'+ page_id +'?fields=comments,reactions')        
        if data_comm.get("comments")!=None:                
            for c in range(0,len(data_comm["comments"]["data"])):
                comm = data_comm["comments"]["data"][c]
                created_time_comm = datetime.strptime(comm.get("created_time").split("T")[0], '%Y-%m-%d')
                if created_time_comm>=query_time_comm:
                    try:
                        insert_facebook(cliente, comm.get("id"), query_time_post, query_time_comm,False)
                        db_reacao.add_reacao_face(cliente,2,comm["message"], 0,int(time.mktime(created_time_comm.timetuple())),str(comm["id"]))
                        print("Registro inserido".center(110,"*"))
                        print("Comentário: " + comm["message"])
                        insert_comment += 1
                    except Exception as e:
                        erros_insert_comment += 1
                        print("Registro com erro".center(110,"*"))
                        print("Comentário: " + comm["message"])
                        print("Erro: " + str(e))  
        if data_comm.get("reactions")!=None:
            for c in range(0,len(data_comm["reactions"]["data"])):
                reac = data_comm["reactions"]["data"][c]
                try:            
                    db_reacao.add_reacao_face(cliente,3,reac["type"], reac_sentimento.get(reac["type"],0),int(time.mktime(datetime.now().timetuple())),str(post["id"])+"_"+str(reac["id"]))
                    print("Registro inserido".center(110,"*"))
                    print("Reação: " + reac["type"])
                    insert_comment += 1
                except Exception as e:
                    erros_insert_comment += 1
                    print("Registro com erro".center(110,"*"))
                    print("Reação: " + reac["type"])
                    print("Erro: " + str(e))    

In [4]:
# Initialize the Graph API with a valid access token (optional,
# but will allow you to do all sorts of fun stuff).
graph = GraphAPI("EAACEdEose0cBAHsoeM9xdKScd5cjp1raRxwE5ZC1ZBmRhlr5UQJVBInONytEGnTjSlDofSXknFdKuj54o3Ayq3E7jZCRP8sNQP4ldZCZCcntKDfQsgsciIPKsWn9cU42OyllcV7ZBuK9NAcatUk9i4kwgPr1LZBwEVOxnMhQfSNYuuTmvEFI036GF1PNjzPWpcZD")

In [74]:
erros_insert_comment = 0
insert_comment=0
get_feed = True
cliente = '1'
query_time_post = datetime.strptime("2018-04-01", '%Y-%m-%d')
query_time_comm = datetime.strptime("2018-04-01", '%Y-%m-%d')
#page_id = '745618718900423'
page_id = 'casariobotequim'
#page_id = 'oleodealegriavv'

insert_facebook(cliente, page_id, query_time_post, query_time_comm,get_feed)

print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

**********************************************Registro com erro***********************************************
Comentário: Jordana Marquess
Erro: Object Reacao[new:66] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
**********************************************Registro com erro***********************************************
Comentário: muito bom !!!
Erro: Object Reacao[new:67] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
**********************************************Registro com erro***********************************************
Comentário: Samba na veia
Erro: Object Reacao[new:68] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
**********************************************Registro com erro***********************************************
Comentário: Franklin Piter, mal Kal Azevedo
Erro: Object Reacao[new:69] cannot be 

KeyboardInterrupt: 